In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:13 http://security.ubuntu.c

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-18 02:03:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.59MB/s    in 0.6s    

2022-10-18 02:03:46 (1.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
cleaned_df = df.dropna()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [8]:
# Create the vine_table. DataFrame
vine_df = cleaned_df.select(["review_id","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

# **Deliverable 2** : Determine Bias of Vine Reviews

In [9]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [10]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve 
# all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
votes_percentage_df = total_votes_df.withColumn('votes_percentage', col("helpful_votes")/col("total_votes")).alias("votes_percentage").filter(col("votes_percentage") >= 0.5)
votes_percentage_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|0.8260869565217391|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|               1.0|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|               1.0|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|0.7924528301886793|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|               1.0|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|               1.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|               0.8|
| RIIGLD8JB7PX8|          1|  

In [11]:
#Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves 
# all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_review_df = votes_percentage_df.filter(col("vine") == "Y")
paid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R184FOUNZZ7KO8|          5|           15|         20|   Y|                N|              0.75|
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|0.8461538461538461|
|R1UYHBYE6790BU|          5|           44|         53|   Y|                N|0.8301886792452831|
|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N|0.9314641744548287|
|R3QDI539WTXKE2|          5|           26|         32|   Y|                N|            0.8125|
| RQTPWY6ND2NRV|          4|           37|         37|   Y|                N|               1.0|
| RQZSTE0KOBY2G|          4|           98|        109|   Y|                N|0.8990825688073395|
| RF2RNZEJO0K1N|          4|  

In [12]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_review_df = votes_percentage_df.filter(col("vine") == "N")
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  votes_percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|0.8260869565217391|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|               1.0|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|               1.0|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|0.7924528301886793|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|               1.0|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|               1.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|               0.8|
| RIIGLD8JB7PX8|          1|  

In [14]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
from pyspark.sql.functions import col,when,count
ratings_df = votes_percentage_df.groupby("vine").agg(count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("5_Star_Reviews_Percentage")).show()

+----+-------------+--------------------+-------------------------+
|vine|Total_Reviews|Total_5_star_Reviews|5_Star_Reviews_Percentage|
+----+-------------+--------------------+-------------------------+
|   Y|         1080|                 454|        42.03703703703704|
|   N|        49659|               23034|        46.38434120703196|
+----+-------------+--------------------+-------------------------+

